In [1]:
import torch
import torch.nn as nn

In [2]:
x=torch.rand(128,32,512)
d_model=512
n_head=8
print(x)

tensor([[[0.8519, 0.4641, 0.4086,  ..., 0.3193, 0.8218, 0.6690],
         [0.9595, 0.3154, 0.9714,  ..., 0.7730, 0.4269, 0.5733],
         [0.1641, 0.4766, 0.5172,  ..., 0.9114, 0.1794, 0.5730],
         ...,
         [0.0469, 0.7299, 0.6926,  ..., 0.9576, 0.9861, 0.3480],
         [0.6076, 0.6442, 0.8933,  ..., 0.2551, 0.0993, 0.9793],
         [0.5506, 0.9161, 0.0407,  ..., 0.0694, 0.5707, 0.8420]],

        [[0.4210, 0.5858, 0.7163,  ..., 0.3907, 0.2523, 0.8049],
         [0.6762, 0.4809, 0.4451,  ..., 0.8268, 0.9438, 0.4585],
         [0.1543, 0.0526, 0.7058,  ..., 0.7519, 0.9232, 0.6365],
         ...,
         [0.5103, 0.2334, 0.8384,  ..., 0.0545, 0.9776, 0.6434],
         [0.8146, 0.4679, 0.0789,  ..., 0.6532, 0.3450, 0.6298],
         [0.0552, 0.6068, 0.3791,  ..., 0.7305, 0.2098, 0.4063]],

        [[0.9523, 0.3587, 0.1386,  ..., 0.3348, 0.5680, 0.0610],
         [0.9646, 0.0879, 0.4080,  ..., 0.1971, 0.0317, 0.2476],
         [0.8919, 0.5665, 0.7947,  ..., 0.4214, 0.0965, 0.

In [3]:
class MutiHeadAttention(nn.Module):
    def __init__(self,d_model,n_head):
        super(MutiHeadAttention,self).__init__()
        self.n_head=n_head
        #注意力头数
        self.d_model=d_model
        #输入输出特征维度
        ##线性映射层，将输入映射为 Q K W
        self.w_q=nn.Linear(d_model,n_head)
        #     Args:
        # in_features: size of each input sample
        # out_features: size of each output sample
        # bias: If set to ``False``, the layer will not learn an additive bias.
        #     Default: ``True``
        #y=xW+b 数学的线性变换  
        self.w_k=nn.Linear(d_model,n_head)
        self.w_v=nn.Linear(d_model,n_head)
        #输出融合层，将多头输出并合并会d_model
        self.w_combine=nn.Linear(d_model,n_head)
        self.softmax=nn.Softmax(dim=-1)

    def forward(self,q,k,v,mask=None):
        batch,time,dimension=q.shape       #batch   序列长度    特征维度
        n_d=self.d_model//self.n_head      #每个头的维度=特征维度/头的个数
        #
        q,k,v=self.w_q(q),self.w_k(k),self.w_v(v)   #q,k,v线性映射成Q,K,V
        q=q.view(batch,time,self.n_head,n_d).permute(0,2,1,3)
        #view从后往前看，把tensor先变成(sum,)，然后从最后一个维度开始分块，依次往前
        #permute就是维度互换    从左往后对齐序列
        k=k.view(batch,time,self.n_head,n_d).permute(0,2,1,3)
        v=v.view(batch,time,self.n_head,n_d).permute(0,2,1,3)
        score=q@k.transpose(2,3)/math.sqrt(n_d)
        #Q维度batch,time,self.n_head,n_d        K^T维度batch,time,n_d,self.n_head
        #transpose 单独两个维度进行互换
        #高纬tensor相乘规则：矩阵的最后两维满足矩阵相乘原则，前面的n-2维相同（或者利用广播机制 可以为1）
        #score batch,time,n_head,n_head
        if mask is not None:
            score=score.masked_fill(mask==0,-100000)#将无效位置填充为-100000 mask==0就是前面提到的padding=0
            score=self.softmax(score)@v
            score=score.permute(0,2,1,3).contiguous().view(batch,time,dimension)
            #score batch,time,n_head,n_head       
            output=self.w_combine(score)
            #self定义的self.w_combine=nn.Linear(d_model,n_head)         融合多头注意力
            return output


attention=MutiHeadAttention(d_model,n_head)



In [5]:
out=attention(x,x,x)
print(out)

Traceback (most recent call last):
  File "/home/sunqingxin/anaconda3/envs/transformer/lib/python3.10/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_vars.py", line 622, in change_attr_expression
    value = eval(expression, frame.f_globals, frame.f_locals)
  File "<string>", line 1, in <module>
NameError: name 'tensor' is not defined


RuntimeError: shape '[128, 32, 8, 64]' is invalid for input of size 32768